# Chapter 04: 환경 구축

## 1. 학습 목표

* Fine-tuning을 위한 최적의 GPU 환경을 확인하고 설정한다.
* `transformers`, `peft`, `bitsandbytes` 등 필수 라이브러리를 설치한다.
* HuggingFace Hub에 로그인하여 모델 다운로드 권한을 확보한다.
* 대용량 모델 관리를 위한 캐시 및 저장 경로를 체계적으로 구성한다.

## 2. 하드웨어 요구사항

LLM Fine-tuning은 막대한 연산 자원을 필요로 한다. 학습 방식에 따른 VRAM 요구량은 다음과 같다.

### 2.1 gpt-oss-20b (20B 파라미터) 기준

| 방식 | 최소 VRAM | 권장 GPU | 비고 |
| --- | --- | --- | --- |
| **Full Fine-tuning** | 160GB+ | 4x A100 (80GB) | 개인이 수행하기 어려움 |
| **LoRA (FP16)** | 48GB+ | 1x A100 (80GB) | 고성능 GPU 필요 |
| **QLoRA (4-bit)** | **24GB+** | **1x RTX 3090/4090** | **소비자용 GPU로 가능 (본 튜토리얼 권장)** |
| **QLoRA (최소)** | 16GB | 1x RTX 4080 / T4 | 배치 사이즈 1, 짧은 시퀀스 길이로 제한적 가능 |

### 2.2 클라우드 GPU 옵션

로컬 GPU가 없다면 클라우드 환경을 이용할 수 있다.

* **Google Colab**: Pro+ 요금제 사용 시 A100 또는 T4 GPU 사용 가능.
* **RunPod / Lambda Labs**: 시간당 저렴한 비용으로 A100/H100 대여 가능.
* **AWS / Azure / GCP**: 엔터프라이즈급 안정성 제공.

## 3. GPU 환경 확인

먼저 현재 환경에서 GPU를 사용할 수 있는지 확인한다.

In [1]:
import torch
import subprocess

def check_gpu_environment():
    """
    현재 시스템의 GPU 상태를 상세히 확인하는 함수다.
    """
    print("=" * 40)
    print("GPU 환경 점검")
    print("=" * 40)

    # 1. CUDA 사용 가능 여부 확인
    if not torch.cuda.is_available():
        print("경고: CUDA를 사용할 수 없다. CPU만으로는 Fine-tuning이 불가능하다.")
        return False

    # 2. GPU 장치 정보 출력
    gpu_count = torch.cuda.device_count()
    print(f"발견된 GPU 개수: {gpu_count}")

    for i in range(gpu_count):
        device_name = torch.cuda.get_device_name(i)
        # VRAM 용량 계산 (GB 단위)
        total_memory = torch.cuda.get_device_properties(i).total_memory / (1024 ** 3)
        print(f"GPU {i}: {device_name} | VRAM: {total_memory:.1f} GB")

    # 3. 소프트웨어 버전 확인
    print(f"\nPyTorch 버전: {torch.__version__}")
    print(f"CUDA 버전: {torch.version.cuda}")

    return True

# nvidia-smi 명령어로 현재 상태(온도, 전력 등) 확인
def print_nvidia_smi():
    try:
        # 쉘 명령어 실행
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        print("\n[nvidia-smi 출력 결과]")
        print(result.stdout)
    except FileNotFoundError:
        print("nvidia-smi 명령어를 찾을 수 없다. 드라이버 설치를 확인해야 한다.")

check_gpu_environment()
print_nvidia_smi()

GPU 환경 점검
발견된 GPU 개수: 8
GPU 0: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 1: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 2: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 3: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 4: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 5: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 6: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB
GPU 7: NVIDIA H100 80GB HBM3 | VRAM: 79.2 GB

PyTorch 버전: 2.6.0+cu126
CUDA 버전: 12.6

[nvidia-smi 출력 결과]
Thu Dec 25 09:01:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                      

## 4. 필수 라이브러리 설치

LLM Fine-tuning을 위해 필요한 핵심 라이브러리들을 설치하고 버전을 확인한다.

### 4.1 핵심 라이브러리 목록

* **transformers**: 모델 로드 및 토크나이징을 위한 표준 라이브러리.
* **peft**: LoRA, QLoRA 등 효율적인 파라미터 학습을 지원한다.
* **bitsandbytes**: 4-bit/8-bit 양자화를 위한 필수 라이브러리 (GPU 전용).
* **accelerate**: 복잡한 분산 학습 및 하드웨어 최적화를 쉽게 처리한다.
* **trl**: SFT, DPO 등 LLM 학습에 특화된 고수준 트레이너를 제공한다.

In [2]:
# 라이브러리 설치 명령어 (주석 해제 후 실행)
# %pip install -q torch torchvision torchaudio
%pip install -q transformers datasets accelerate peft trl bitsandbytes
%pip install -q wandb

def check_library_versions():
    """
    주요 라이브러리의 설치 여부와 버전을 확인한다.
    """
    packages = [
        "torch", "transformers", "datasets", "accelerate",
        "peft", "trl", "bitsandbytes"
    ]

    print("\n라이브러리 버전 확인:")
    print("-" * 30)

    for package in packages:
        try:
            module = __import__(package)
            version = getattr(module, "__version__", "알 수 없음")
            print(f"{package:<15}: {version}")
        except ImportError:
            print(f"{package:<15}: 설치되지 않음 (설치 필요)")

check_library_versions()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

라이브러리 버전 확인:
------------------------------
torch          : 2.6.0+cu126


/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers   : 4.57.3
datasets       : 4.4.2
accelerate     : 1.12.0
peft           : 0.18.0
trl            : 0.26.2
bitsandbytes   : 0.49.0


## 5. HuggingFace 설정

많은 최신 모델(LLaMA 3, Gemma 등)은 HuggingFace Hub의 접근 권한 승인이 필요하다. 토큰을 사용하여 로그인한다.

In [2]:
from huggingface_hub import login, HfApi

def setup_huggingface(token=None):
    """
    HuggingFace Hub에 로그인하고 상태를 확인한다.
    token이 없으면 대화형 로그인을 시도한다.
    """
    print("\nHuggingFace 로그인 설정")

    # 토큰이 제공되면 비대화형 로그인, 아니면 대화형 로그인
    if token:
        login(token=token)
    else:
        # 주석 해제 후 토큰 직접 입력 가능
        # login()
        print("참고: 코드로 로그인하려면 login(token='hf_...')을 사용한다.")

    # 로그인 상태 확인
    try:
        api = HfApi()
        user_info = api.whoami()
        print(f"로그인 성공: {user_info['name']} (Type: {user_info['type']})")
    except Exception as e:
        print(f"로그인 상태 확인 실패: {e}")
        print("HuggingFace 설정 페이지에서 Access Token을 발급받아 입력해야 한다.")

# 실제 사용 시 토큰 입력
setup_huggingface("hf_허깅페이스에서 발급 받은 Key Token을 붙혀 넣는다")


HuggingFace 로그인 설정
로그인 성공: nowave (Type: user)


## 6. 캐시 및 저장 경로 설정

대규모 모델은 용량이 크기 때문에 기본 경로(홈 디렉토리)에 저장하면 용량 부족 문제가 발생할 수 있다. 넉넉한 공간이 있는 별도 경로로 설정하는 것이 좋다.

In [ ]:
import os

# 프로젝트 루트 경로 설정
PROJECT_ROOT = "./gpt-oss-20b-finetuning"
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "outputs")

# HuggingFace 캐시 경로 변경 (필요 시)
# os.environ["HF_HOME"] = "/path/to/large/disk/huggingface"

def setup_directories():
    """
    프로젝트에 필요한 디렉토리 구조를 생성한다.
    """
    dirs = [MODELS_DIR, DATA_DIR, OUTPUT_DIR]

    print("\n디렉토리 설정:")
    for d in dirs:
        os.makedirs(d, exist_ok=True)
        print(f"생성/확인 완료: {d}")

    print(f"현재 HF_HOME: {os.getenv('HF_HOME', '기본 경로 (~/.cache/huggingface)')}")

setup_directories()

## 7. 모델 로드 테스트 (환경 검증)

모든 설정이 올바른지 확인하기 위해 작은 모델(`gpt2`)을 로드하여 간단한 추론을 실행해본다. `bitsandbytes`가 정상 작동하는지도 이 단계에서 확인할 수 있다.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def test_environment_with_small_model():
    """
    작은 모델(GPT-2)을 로드하여 라이브러리와 GPU가 정상 작동하는지 검증한다.
    """
    print("\n환경 검증 테스트 (GPT-2)")
    test_model_name = "gpt2"

    # 1. 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(test_model_name)

    # 2. 모델 로드 (GPU 사용)
    model = AutoModelForCausalLM.from_pretrained(
        test_model_name,
        device_map="auto"
    )

    # 3. 추론 테스트
    input_text = "Fine-tuning is"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=10)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"입력: {input_text}")
    print(f"출력: {result}")
    print("검증 성공: PyTorch, Transformers, GPU가 정상 작동한다.")

# 테스트 실행 (오류 발생 시 설치 과정 재확인 필요)
test_environment_with_small_model()


환경 검증 테스트 (GPT-2)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


입력: Fine-tuning is
출력: Fine-tuning is a very important part of the process.


검증 성공: PyTorch, Transformers, GPU가 정상 작동한다.


## 8. W&B 설정 (선택 사항)

Weights & Biases(W&B)는 학습 과정을 실시간으로 모니터링하고 시각화하는 강력한 도구다. 필수는 아니지만 강력히 권장한다.
- https://wandb.ai 에서 회원가입

In [1]:
import wandb

def setup_wandb():
    """
    W&B 로그인 안내를 출력한다.
    """
    print("\nW&B(Weights & Biases) 설정")
    print("1. https://wandb.ai/authorize 에서 API 키 복사")
    print("2. wandb.login() 실행 후 키 입력")
    wandb.login()

setup_wandb()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



W&B(Weights & Biases) 설정
1. https://wandb.ai/authorize 에서 API 키 복사
2. wandb.login() 실행 후 키 입력


wandb: Currently logged in as: kubwai to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 9. 요약

이 챕터에서는 Fine-tuning을 위한 물리적, 소프트웨어적 기반을 마련했다.

1. **GPU 확인**: QLoRA 학습을 위해 최소 24GB VRAM(RTX 3090/4090)을 권장한다.
2. **라이브러리**: `peft`, `bitsandbytes`, `trl` 등 최신 Fine-tuning 스택을 설치했다.
3. **검증**: 작은 모델 구동 테스트를 통해 환경이 정상임을 확인했다.

이제 모든 준비가 완료되었다. 다음 챕터부터는 실제 `gpt-oss-20b` 모델을 로드하고 본격적인 Fine-tuning 과정을 시작한다.

다음 챕터는 **Chapter 05: gpt-oss-20b 모델 로딩**으로, 대규모 모델을 4-bit로 양자화하여 메모리에 올리는 방법을 다룬다.

---

**Chapter 04 생성이 완료되었다. 다음 챕터(Chapter 05)로 진행할까요?**